<a href="https://colab.research.google.com/github/calicartels/Explainable-Techniques-LIME/blob/main/Explainable_Techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explainable Techniques : LIME

Cloning the repo:

In [ ]:
!git clone https://github.com/calicartels/Explainable-Techniques-LIME.git

First, we will install the necessary libraries. In addition to numpy and pandas, we will use the following libraries:


*   scikit-learn (Model, ICE plots)
*   lime (LIME)


In [ ]:
!pip install numpy==1.25.2 pandas==2.0.3 matplotlib==3.7.1 scikit-learn==1.2.2 lime==0.2.0.1 anchor-exp==0.0.2.0 shap==0.45.1 xgboost==1.7.5

In [ ]:
# Data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Models
from keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions
from keras.preprocessing import image
import tensorflow as tf

# XAI
import lime
from lime import lime_image
from skimage.segmentation import mark_boundaries

np.random.seed(1)

In [ ]:

# Enabling GPU usage
device_name = tf.test.gpu_device_name()
if device_name:
    print(f"Using GPU: {device_name}")
else:
    print("No GPU found. Using CPU.")

# Loading pre-trained InceptionV3 model + weights
model = InceptionV3(weights='imagenet')

# Loading an image (using a sample class here, dalmatian)
img_path = "/content/Explainable-Techniques-LIME/dalmation.jpeg"
img = image.load_img(img_path, target_size=(299, 299))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

# Predicting using InceptionV3
preds = model.predict(img_array)
print('Predicted:', decode_predictions(preds, top=3)[0])

# LIME Image explanation
explainer = lime_image.LimeImageExplainer()


In [ ]:
# Explaining prediction for the image
explanation = explainer.explain_instance(img_array[0], model.predict, top_labels=5, hide_color=0, num_samples=1000)

# Visualizing the explanation
temp, mask = explanation.get_image_and_mask(
    explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=False
)


plt.imshow(mark_boundaries(img_array[0] / 2 + 0.5, mask))  # Rescaling to original image range
plt.show()

In [ ]:
# Visualizing superpixels
temp, mask = explanation.get_image_and_mask(
    explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=False
)

# Displaying original image with only the important superpixels highlighted
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

# Original image with LIME explanation
ax[0].imshow(mark_boundaries(img_array[0] / 2 + 0.5, mask))
ax[0].set_title("LIME Explanation")

# Superpixels alone (highlighted with color)
ax[1].imshow(mask, cmap='gray')
ax[1].set_title("Highlighted Superpixels")

plt.show()

	1.	Left Side (LIME Explanation): The original image with important regions (superpixels) overlaid.
	2.	Right Side (Superpixels Alone): A grayscale image where only the selected superpixels are highlighted, making it easier to understand which parts of the image the model focused on.

This visualization provides another layer of insight by isolating the important regions in the image that LIME considers influential for the prediction.


## Why LIME?

LIME (Local Interpretable Model-agnostic Explanations) is a great choice for image models like InceptionV3 because:

	•	It explains individual predictions by highlighting regions of the image that were most influential in the model’s decision.
	•	It works by perturbing the input image and observing how changes impact the model’s predictions, offering visual insights into the black-box model.

Strengths:

	•	Local interpretability: LIME helps understand which parts of the image influenced the model’s decision for that specific image.
	•	Model-agnostic: It works with any model, including deep neural networks like Inception.
	•	Visual explanations: It highlights important image regions, providing clear visual feedback.

Limitations:

	•	Approximate explanation: The explanation is based on a linear model fitted locally to perturbations of the image, so it might not capture complex decision boundaries accurately.
	•	Perturbation-based: LIME’s reliance on random perturbations can make it computationally expensive, especially with high-resolution images.

Potential Improvements:

	•	Better segmentations: Improving the way LIME segments images could lead to more accurate explanations.
	•	Hybrid methods: Combining LIME with methods like SHAP could yield more theoretically sound explanations.
